<a href="https://colab.research.google.com/github/tanhhkhcndn/mobilenet-transfer-learning/blob/main/mobilenetSSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training MobileNetSSD Object Detection on a Custom Dataset
## Overview
This notebook walks through how to train a MobileNet object detection model using the TensorFlow 1.5 Object Detection API.

In this specific example, we'll training an model to detect thief in edge devices.
## **Our Data**

We'll be using an image dataset from videos. Our dataset contains 614 images.
We create two datasets in Roboflow: `train`, `test` and `valid`. Use Roboflow to generate TFRecords for each, replace their URLs in this notebook, and you're able to train on your own custom dataset.

## **Our Model**

We'll be training a MobileNetSSDv2 (single shot detector). This specific model is a one-short learner, meaning each image only passes through the network once to make a prediction, which allows the architecture to be very performant for low performace hardware.

The model arechitecture is one of many available via TensorFlow's [model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models).

As a note, this notebook presumes TensorFlow 1.5 as TensorFlow 2.0 has yet to fully support the object detection API.

## **Training**

Google Colab provides free GPU resources. Click "Runtime" → "Change runtime type" → Hardware Accelerator dropdown to "GPU."

Colab does have memory limitations, and notebooks must be open in your browser to run. Sessions automatically clear themselves after 12 hours.


## Step 1: Clone TensorFlow2 object detection API folder to your colab:

In [1]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 4065, done.
remote: Counting objects: 100% (4065/4065), done.
remote: Compressing objects: 100% (3089/3089), done.
remote: Total 4065 (delta 1186), reused 1941 (delta 916), pack-reused 0
Receiving objects: 100% (4065/4065), 54.71 MiB | 23.91 MiB/s, done.
Resolving deltas: 100% (1186/1186), done.


## Step 2: Install object detection API in colab
pip install tensorflow[and-cuda]==2.13.0 --> fix error
module 'tensorflow.python.ops.control_flow_ops' has no attribute 'case'

In [ ]:
# Install the Object Detection API
%%bash
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .
pip install tensorflow[and-cuda]==2.13.0

## Step 3: Import the TensorFlow libraries to colab

In [3]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

## Step 4: Run this command to test if object detection API is installed properly:

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

## Step 5: Prepare custom set of images for training. I’m using Roboflow in this example to create the TFRecord file from the training images.
Downloaded files will go to /content/train/ folder in colab environment.

In [ ]:
%cd /content

#Download Training set from git by cloning rep:
import os
import pathlib
# Clone the training set repository if it doesn't already exist
if "ThiefTFData" in pathlib.Path.cwd().parts:
  while "ThiefTFData" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('ThiefTFData').exists():
  !git clone --depth 1 https://github.com/tanhhkhcndn/mobilenet-transfer-learning/
  %cd /content/mobilenet-transfer-learning/ThiefTFData
#  !unzip Pickachu.v1.tfrecord.zip -d /content/

#NOTE: Update these TFRecord names to your files containing training set!
#Also, Update relevant rows:in training config file "ssd_mobilenet_v2_320x320_coco17_tpu-8.config"
#label_map_path,input_path

test_record_fname = '/content/mobilenet-transfer-learning/ThiefTFData/test/Thief.tfrecord'
train_record_fname = '/content/mobilenet-transfer-learning/ThiefTFData/train/Thief.tfrecord'
label_map_pbtxt_fname = '/content/mobilenet-transfer-learning/ThiefTFData/train/Thief_label_map.pbtxt'

## Step 6: Set configuration parameters for training the model:

In [6]:
#You can change chosen model to deploy different models available in the TF2 object detection zoo
MODELS_CONFIG = {
    'ssd_mobilenet_v2_320x320_coco17': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
        'batch_size': 16
    }
}
chosen_model = 'ssd_mobilenet_v2_320x320_coco17'
num_steps = 1800 #40000 #The more steps, the longer the training. Increase if your loss function is still decreasing and validation metrics are increasing.
num_eval_steps = 500 #Perform evaluation after so many steps
model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] #if you can fit a large batch in memory, it may speed up your trainin#g

## Step 7: Load trained model weights from TensorFlow site, and the custom configuration file which I’ve prepared for this example:

In [ ]:
#Download pretrained weights
%mkdir /content/deploy/
%cd /content/deploy/
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()
#Shorten the folder name,because long file paths are not yet supported :(
os.rename('ssd_mobilenet_v2_320x320_coco17_tpu-8','mobilnetv2')

In [ ]:
%cd /content/deploy
download_config = '!wget https://raw.githubusercontent.com/tanhhkhcndn/mobilenet-transfer-learning/main/ssd_mobilenet_v2_320x320_coco17_tpu-8.config'
!wget {download_config}

In [9]:
#Prepare frozen model for retraining
fine_tune_checkpoint = '/content/deploy/mobilnetv2/checkpoint/ckpt-0'
pipeline_file = '/content/deploy/ssd_mobilenet_v2_320x320_coco17_tpu-8.config'
model_dir = '/content/training/'


def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)

## Step 8: Run the training process:
From my experience, loss function result with a value lower than0.2 is enough for a demo application.

In [10]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

INFO:tensorflow:Step 1800 per-step time 5.522s
I1201 07:45:57.621553 140310429614080 model_lib_v2.py:705] Step 1800 per-step time 5.522s
INFO:tensorflow:{'Loss/classification_loss': 0.99283457,
 'Loss/localization_loss': 0.6150839,
 'Loss/regularization_loss': 4.7656126,
 'Loss/total_loss': 6.373531,
 'learning_rate': 0.733333}
I1201 07:45:57.621988 140310429614080 model_lib_v2.py:708] {'Loss/classification_loss': 0.99283457,
 'Loss/localization_loss': 0.6150839,
 'Loss/regularization_loss': 4.7656126,
 'Loss/total_loss': 6.373531,
 'learning_rate': 0.733333}


## Step 9: Step 9: Save the retrained model:

Exported model will be stored at output_directory. In this example the model will be exported to /content/fine_tuned_model.
save fine_tune_model to local drive by zip

In [ ]:
#run conversion script to save the retrained model:
#Saved model will be in saved_model.pb file:

import re
import numpy as np

output_directory = '/content/fine_tuned_model'

#place the model weights you would like to export here
last_model_path = '/content/training/'
print(last_model_path)
!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}

In [ ]:
!zip -r /content/fine_tuned_model.zip /content/fine_tuned_model

## Step 10: Testing our retrained model:
Step A: Import the images you want to test to the /content/test folder.

In [ ]:
%mkdir /content/test/
%cd /content/test/
!gdown 11vqszjiGxdgogvJ_lJ7wOrgUUaTYYPeC

Step B: Import visualization libraries and object detection functions:

In [18]:
import os
import glob
import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

Step C: Import our retrained model, its last training checkpoint and map object labels. For this example I chose ckpt-2 (second checkpoint).

In [ ]:
#Recover our saved model with the latest checkpoint:
pipeline_config = pipeline_file
#Put the last ckpt from training in here, don't use long pathnames:
model_dir = '/content/training/ckpt-1'
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore last checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
#ckpt.restore(os.path.join(model_dir))
ckpt.restore(model_dir)

#Function perform detection of the object on image in tensor format:
def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""
    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

#Define function which performs detection:
detect_fn = get_model_detection_function(detection_model)


#map labels for inference decoding
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

Step D: Perform object detection on test images:

In [ ]:
#run detector on test image
#it takes a little longer on the first run and then runs at normal speed.
import random

#Define utility functions for presenting the results:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.
  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.
  Args:
    path: the file path to the image
  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)


#Place your test images here:
image_path = '/content/test/thiefTest.jpg'

#Store test images in nmpy array:
image_np = load_image_into_numpy_array(image_path)

#Convert images to tensor form:
input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)

#Perform detection on the image in tensor format:
detections, predictions_dict, shapes = detect_fn(input_tensor)

#Visualize the detection boxes on the image:
label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=0.5,#0.5,#0.5
      agnostic_mode=False,
)

plt.figure(figsize=(12,16))
plt.imshow(image_np_with_detections)
plt.show()